In [1]:
import opendatasets as od

od.download('https://www.kaggle.com/datasets/averkij/tennessee-eastman-process-simulation-dataset')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading tennessee-eastman-process-simulation-dataset.zip to .\tennessee-eastman-process-simulation-dataset


100%|██████████| 1.28G/1.28G [13:57<00:00, 1.64MB/s]


In [ ]:
import pyreadr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
# Only Training data is used

df_FaultFree = pyreadr.read_r(r'F:\TEP_Data\The TEP Dataset\TEP_FaultFree_Training.RData')['fault_free_training']
df_Faulty = pyreadr.read_r(r'F:\TEP_Data\The TEP Dataset\TEP_Faulty_Training.RData')['faulty_training']

In [ ]:
DF = pd.concat([df_FaultFree,df_Faulty]) # Join both fault free and faulty dataset
DF.head()